# Understanding Prompt Templates

In [1]:
# Imports
import os
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, SystemMessage, HumanMessage
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

api_key = os.getenv("OPENAI_API_KEY")

### With No Input Variables

In [2]:
# An example of Prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()

'Tell me a fact'

### With a Single Input Variable

In [3]:
one_input_prompt = PromptTemplate(input_variables=["topic"], template="Tell me a fact about {topic}")
one_input_prompt.format(topic="Mars")

'Tell me a fact about Mars'

### Multiple Input Variables

In [4]:
multiple_input_prompt = PromptTemplate(input_variables=["topic", "level"],
                                       template="Tell me about a fact about {topic} for a student at {level} level")
multiple_input_prompt.format(topic='Mars', level='8th grade')

'Tell me about a fact about Mars for a student at 8th grade level'

### Chat Model Templates

Chat models require a list of chat messages called a prompt, which is different from a raw string that you would input into a language model. Each message in the prompt is associated with a role, such as AI, human, or system.
For instance, when using the OpenAI Chat Completion API, a chat message can be assigned the role of AI, human, or system. The model is designed to pay closer attention to instructions provided in system chat messages.

To simplify the process of constructing and working with prompts, LangChain offers various prompt templates. It is highly recommended to utilize these chat-related prompt templates instead of PromptTemplate when interacting with chat models. This will allow you to fully harness the potential of the underlying chat model and enhance your experience.

We will favor these models in the course due to upcoming changes in the OpenAI ecosystem where chat agents will be favored over text completion models.

In [5]:
system_template = "You are an AI recipe Assistant that specialises in {dietary_preference} dishes that can prepared in {cooking_time}" 
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# to check the input variables for the prompt template
system_message_prompt.input_variables

['cooking_time', 'dietary_preference']

In [6]:
human_template = "{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
human_message_prompt.input_variables

['recipe_request']

In [7]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt.input_variables

['cooking_time', 'dietary_preference', 'recipe_request']

In [8]:
# Now we can get a chat completion from the formatted messages
chat_prompt.format_prompt(cooking_time='15 min', dietary_preference='Non-Veg', recipe_request='chicken salad').to_messages()

[SystemMessage(content='You are an AI recipe Assistant that specialises in Non-Veg dishes that can prepared in 15 min', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='chicken salad', additional_kwargs={}, response_metadata={})]

#### Now we make the LLM Call

In [9]:
chat = ChatOpenAI(openai_api_key = api_key)

request = chat_prompt.format_prompt(cooking_time='15 min', dietary_preference='Non-Veg', recipe_request='chicken salad').to_messages()

result = chat(request)

/var/folders/ny/r3wsv_bd0bs1_vcy4pb5q9pr0000gn/T/ipykernel_62132/2825063962.py:5: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chat(request)


In [10]:
print(result.content)

Ingredients:
- 2 cups cooked chicken breast, diced
- 1/2 cup mayonnaise
- 1/4 cup Greek yogurt
- 1 tablespoon Dijon mustard
- 1 celery stalk, diced
- 1/4 cup red onion, diced
- 1/4 cup dried cranberries or raisins
- Salt and pepper to taste
- Lettuce leaves for serving

Instructions:
1. In a mixing bowl, combine the mayonnaise, Greek yogurt, and Dijon mustard.
2. Add the diced chicken, celery, red onion, and cranberries to the bowl.
3. Mix everything together until well combined, then season with salt and pepper to taste.
4. Serve the chicken salad on a bed of lettuce leaves.
5. Enjoy your quick and easy chicken salad!


#### One More Example - All Encapsulated within a function

In [21]:
def travel_idea (interest: str, duration: int):
    '''
    interest : destination where you want to go
    durartion : no of days you want your itinerary to be
    '''

    # SYSTEM
    system_template = "You are an AI Travel Agent that helps people to arrange detailed itinerary for the destination {interest} for the {duration} days"
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    # HUMAN REQUEST
    human_template = "{human_request}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    # COMPILE TO CHAT
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    # INSERT VARIABLES
    request = chat_prompt.format_prompt(interest=interest, duration=duration, human_request="Please give a detailed day wise vacation itinerary for").to_messages()
    # CHAT REQUEST
    chat = ChatOpenAI(openai_api_key=api_key)
    result = chat(request)
    return result

In [23]:
#print(travel_idea('Manali', 5))
#result = travel_idea('Manali', 5)

result = travel_idea('Manali', 5)
print(result.content)

**Day 1: Arrival in Manali**

- Arrive in Manali and check into your hotel/resort.
- Relax and freshen up after your journey.
- In the evening, take a leisurely stroll around the Mall Road, a bustling street with shops, cafes, and restaurants.
- Enjoy a delicious dinner at a local restaurant.
- Overnight stay at the hotel.

**Day 2: Rohtang Pass Excursion**

- Have an early breakfast and head towards Rohtang Pass, a high mountain pass known for its scenic beauty.
- Enjoy activities like skiing, snowboarding, and snowball fights in the snow (if visiting during the snow season).
- Visit the Solang Valley for paragliding and other adventure sports (seasonal).
- Return to Manali in the evening and relax at the hotel.
- Dinner and overnight stay at the hotel.

**Day 3: Manali Local Sightseeing**

- After breakfast, visit the Hadimba Devi Temple, a wooden temple surrounded by cedar forests.
- Explore the Vashisht Hot Water Springs and Temple for a relaxing dip in the natural hot springs.
- V

In [24]:
result = travel_idea('Turkey', 9)
print(result.content)

Day 1: Arrival in Istanbul
- Arrive in Istanbul and check into your hotel
- Visit the iconic Hagia Sophia, Blue Mosque, and Topkapi Palace
- Explore the vibrant Grand Bazaar for shopping and dining
- Enjoy a traditional Turkish dinner

Day 2: Istanbul
- Take a Bosphorus cruise to see Istanbul from the water
- Visit the Spice Bazaar and try Turkish street food
- Explore the Chora Church and Suleymaniye Mosque
- Relax in a traditional Turkish hammam (spa)

Day 3: Cappadocia
- Fly to Cappadocia in the morning
- Visit the Goreme Open-Air Museum and see the fairy chimneys
- Explore the underground cities of Derinkuyu or Kaymakli
- Watch the sunset over the valleys from a hot air balloon (optional)

Day 4: Cappadocia
- Enjoy a hot air balloon ride at sunrise (optional)
- Visit the Selime Monastery and Ihlara Valley for hiking
- Explore the town of Uchisar and its rock castle
- Experience a traditional Turkish dinner with a folklore show

Day 5: Pamukkale
- Travel to Pamukkale by bus or priva